# COMP 561 Final Project

## To Do:
- incorporate DNA physical data into classifier
- implement prediction using position weight matrix to compare to as a baseline

## Objectives
1. Identifying a set of bound and non‐bound DNA sequences for a given TF based on existing experimental data.
2. Calculating the DNA physical properties of each sequence.
3. Training a machine learning classifier to distinguish between bound and unbound sites.

## My Interpretation
1. Find bound and non-bound regions using the publicly available data
2. Use GBshape to identify the physical properties of each bound and non-bound region
3. Train a machine learning classifier using physical properties as features and bound vs non-bound as labels

## We have:
- human genome assembly hg19
- active regulatory regions of GM12878
- TF binding sites
- position weight matrix

## Questions
- why do we need the entire human genome?
    - end goal is to use the classifier to identify binding regions given a sequence and DNA physical properties
- what do we use the position weight matrix for?
    - we can compare the performance of this to our machine learning classifier
- what else?
    - train a classifier on just the sequence data and try to use it to identify transcription factors
   
## Assumptions and Decisions:
- we will use a single chromosome and a single transcription factor. this reduces the size of the problem in a scalable way (if this succeeds then it is safe to say that the approach should work on the larger problem). 
- how should we represent 'N' in a sequence (which means that it can be any nucleotide)?

## Approach

### Data Generations
The data for our classifier will be binding sites on a single chromosome for a single transcription factor. We will select one of the transcription factors that is more common so that we have more data points (~15 000). When considering physical features of DNA, we will select certain features available online, and will most likely restrict the features to simpler properties (ie. not including second order). We will select 14 000 data points from binding sites and label them as class 1. For non-binding sites, we will iterate through regulatory regions and randomly select sequences of 15 nucleotides that are known to be non-binding sites. We considered iterating through  sequence in the genome but the presence of repeats and unspecified nucleotides made that difficult. Plus, it seems more realistic for scientists to scan regulatory regions in search of binding sites instead of scanning the entire genome (which is expensive, but may yield interesting results).

### Data Representation
We will represent each data point as a multidimensional array. The simplest point will be a vector of nucleotides, but we will likely also include other information (ie. physical properties) at each nucleotide.

In [1]:
# variables
chrom='chr1'
tf_name='PAX5'
ratio=0.8 # ratio of binding and non binding sites selected for use as data
physical=False # consider physical data or not

## 1. Identifiying bound/non-bound sequences

In [2]:
# input file: set of genomic regions that are active regulatory regions in GM12878
# a regulatory region is characterized by: chromosome #, start nt, end nt, ID (ex. chr2.3 is the 3rd site on chr2)
reg_regions = []
with open('data/wgEncodeRegTfbsClusteredV3.GM12878.merged.bed', 'r') as file:
    for line in file:
        region = line.rstrip().split('\t')
        
        # only take regions on decided chromsome
        if region[0] != chrom:
            continue
        reg_regions.append(region)

In [3]:
reg_regions[0]
# len(reg_regions)

['chr1', '237550', '237989', 'chr1.1']

In [4]:
# input file: set of genomic coordinates of transcription factor binding sites for several transcription factors
# we chose to look at a single TF: 'CTCF'

# binding_sites is all bound sequences
binding_sites = []

# count frequencies of each tf
tf_freq = {}

with open ('data/factorbookMotifPos.txt', 'r') as file:
    for line in file:
        region = line.rstrip().split('\t')[1:]
        # only take regions on chromosome 1
        if region[0] != chrom:
            continue
            
        key = region[3]
        if key in tf_freq:
            tf_freq[key] += 1
        else:
            tf_freq[key] = 1
            
        if key == tf_name:        
            binding_sites.append(region)

In [5]:
tf_freq

{'UA1': 767,
 'CTCF': 14648,
 'NFY': 2653,
 'FOXA': 4950,
 'RUNX1': 2792,
 'UAK25': 2678,
 'UAK26': 1149,
 'UAK27': 1705,
 'v-Maf': 4302,
 'USF': 4778,
 'BHLHE40': 1609,
 'HNF4': 2487,
 'RXRA': 1190,
 'AP1': 15471,
 'NFE2': 2281,
 'MYC': 2883,
 'NRF1': 3987,
 'PAX5': 1441,
 'YY1': 2557,
 'UAK17': 791,
 'UAK18': 473,
 'SP1': 8720,
 'v-JUN': 3030,
 'CREB-ext': 811,
 'EBF1': 4181,
 'ZNF143-ext': 1801,
 'EGR1': 7011,
 'UAK42': 14428,
 'RFX5': 1055,
 'UA6': 90,
 'UAK52': 887,
 'UAK30': 974,
 'CEBPB': 5596,
 'ELF1': 2456,
 'NFKB1': 2680,
 'MAX': 2859,
 'UAK41': 635,
 'ZNF263': 7595,
 'UA2': 665,
 'GABP': 1645,
 'E2F4': 5039,
 'UAK21': 162,
 'NR2C2': 242,
 'TAL1': 1378,
 'ZEB1': 325,
 'GATA1-ext': 2033,
 'UAK36': 696,
 'AP2': 3167,
 'TEAD4': 1665,
 'UA7': 240,
 'CTCF-ext': 2364,
 'POU2F2': 540,
 'MEF2': 890,
 'UAK61': 340,
 'TCF12': 2394,
 'ZNF281': 1702,
 'UA3': 3275,
 'UAK29': 1412,
 'GATA1': 3542,
 'GATA3': 821,
 'UA9': 870,
 'TCF3': 1322,
 'ESR1': 1296,
 'E2F1': 3601,
 'STAT1': 2351,
 'BA

In [6]:
# binding_sites[-1]
len(binding_sites)

1441

In [7]:
# check that the every binding site has the same length

len_sites = {}
for site in binding_sites:
    key = int(site[2]) - int(site[1])
    if key in len_sites:
        len_sites[key] += 1
    else:
        len_sites[key] = 1

In [8]:
len_sites

{16: 1441}

In [9]:
non_binding = []
k=0
start=0
for region in reg_regions:
    # print to track progress
    k+=1
    if k % 10 == 0 or k == len(reg_regions):
        print('\r', end='')
        print(str(k) + '/' + str(len(reg_regions)), end='')
    
    # check if the region contains a binding site
    '''
    logic: for any given region, the first potential TF binding site cannot be before the first potential TF 
    binding site of the previous region. thus we store the first potential site as new_start and update start 
    at the end of each iteration
    '''
    
    new_start=-1
    contains=False
    for i in range(start, len(binding_sites)):
        if region[0] != binding_sites[i][0] or int(binding_sites[i][1]) < int(region[1]):
            continue
        # this code only met if start of TF binding is >= to start of region
        else:
            # new_start is only -1 once -> only updated the first time TF binding >= start of region
            if new_start == -1:
                new_start = i
            # if this condition is met, there is no overlap between TF binding and region
            if int(binding_sites[i][1]) > int(region[2]):
                break;
            # full overlap
            if int(binding_sites[i][1]) >= int(region[1]) and int(binding_sites[i][2]) <= int(region[2]):
                contains=True;
                break;
    if not contains:
        non_binding.append(region)
    start=new_start

7404/7404

In [10]:
len(non_binding)

6746

In [11]:
with open('data/non_binding_regions.txt', 'w') as f:
    for region in non_binding:
        f.write(region[0] + '\t' + region[1] + '\t' + region[2] + '\n')

## 1.1 Process Binding Sites

In [12]:
# input file: sequence of chromosome 1
chr_seq = ""
chr_lines = []
with open('data/'+chrom+'.fa', 'r') as file:
    next(file)
    chr_lines = file.read().splitlines()
chr_seq = ''.join(chr_lines).upper()

In [13]:
chr_seq[16245:16260]

'GCCAGCAGAGGGGTT'

In [14]:
def complement(sequence):
    '''
    Takes a sequence of all capital letters
    '''
    s=[]
    for nt in sequence:
        if nt == 'A':
            s.append('T')
        elif nt == 'T':
            s.append('A')
        elif nt == 'G':
            s.append('C')
        elif nt == 'C':
            s.append('G')
    complement=''.join(s)
    return complement[::-1]

In [15]:
print(binding_sites)

[['chr1', '710609', '710625', 'PAX5', '1.63', '+'], ['chr1', '762858', '762874', 'PAX5', '1.72', '+'], ['chr1', '805268', '805284', 'PAX5', '1.82', '-'], ['chr1', '840170', '840186', 'PAX5', '1.82', '-'], ['chr1', '894704', '894720', 'PAX5', '1.87', '-'], ['chr1', '894719', '894735', 'PAX5', '1.7', '-'], ['chr1', '894833', '894849', 'PAX5', '1.69', '-'], ['chr1', '936172', '936188', 'PAX5', '1.61', '+'], ['chr1', '937329', '937345', 'PAX5', '1.78', '-'], ['chr1', '937823', '937839', 'PAX5', '2.03', '-'], ['chr1', '986144', '986160', 'PAX5', '1.91', '+'], ['chr1', '993443', '993459', 'PAX5', '1.97', '+'], ['chr1', '995002', '995018', 'PAX5', '2.07', '+'], ['chr1', '995015', '995031', 'PAX5', '1.95', '+'], ['chr1', '995027', '995043', 'PAX5', '1.63', '+'], ['chr1', '995040', '995056', 'PAX5', '1.82', '+'], ['chr1', '999790', '999806', 'PAX5', '1.76', '+'], ['chr1', '1005980', '1005996', 'PAX5', '2.1', '+'], ['chr1', '1051634', '1051650', 'PAX5', '2.07', '+'], ['chr1', '1051693', '1051709

In [16]:
with open('data/binding_sites.txt', 'w') as f:
    for region in binding_sites:
        f.write(region[0] + '\t' + region[1] + '\t' + region[2] + '\n')

In [17]:
b_seq={}
y_b=[]

for site in binding_sites:
    seq = chr_seq[int(site[1]): int(site[2])]
    if site[5] == '-':
        b_seq[int(site[1])] = complement(seq)
    else:
        b_seq[int(site[1])] = seq
    y_b.append(1)

In [18]:
b_seq

{710609: 'ATTAGCCAGGCGTGAA',
 762858: 'CTCACCCGAGCGGACC',
 805268: 'CGCGGGAGGGCGTGCC',
 840170: 'GGCGGCGGCGCGAGGA',
 894704: 'CGCGGCGGGGCGGGGA',
 894719: 'CTCGGCTGGGCGTGGC',
 894833: 'GGGCGCCGAGCGGGAG',
 936172: 'CCCCGCAGAGCAGGGC',
 937329: 'GGGAGGGACGCGGGAC',
 937823: 'GGCAACAGAGCGAGAC',
 986144: 'CTCACTGGAGCGAAAC',
 993443: 'GGCGACAGAGCGCGAC',
 995002: 'GGCCGGGGGGCGTGGC',
 995015: 'GGCCTCGGGGCGTGGC',
 995027: 'TGGCCCGGGGCGTGGC',
 995040: 'GGCATCGGGGCGAGGC',
 999790: 'GGCGGCGAGCTGGGGC',
 1005980: 'GGCCATGGGGCGTGAC',
 1051634: 'GGCGCCGGGGCGGGGC',
 1051693: 'GTCAGGGACCCGAGAC',
 1057543: 'GGCACCCAGGCAGAGA',
 1120595: 'TGCGCTGAAGTGTGAC',
 1136844: 'GGCGGCGGAGCGGGCT',
 1136966: 'GTCAGCCCCGCGGGAC',
 1137011: 'GGGGCCGGTGCGGGAC',
 1206436: 'GCCAGTGATGTGTGAC',
 1209217: 'GCTCGCGGGGCGGAAC',
 1219411: 'AGCACCGAAGCATGGA',
 1243270: 'GCGGCGCGGGCGGGAC',
 1243321: 'CGGGGCGGGGCGCGGC',
 1243326: 'ACAGGCGGGGCGGGGC',
 1243345: 'CGCCGGGGAGCGTCGC',
 1243411: 'GGCGGCGGACCTTGGC',
 1243449: 'CGCCGGGGAGCGTCGC

In [19]:
y_b[90:100]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

## 1.2 Process Non-Binding Sites

In [20]:
nb_seq={}
length=len(next(iter(b_seq.values())))
        
for region in non_binding:
    for i in range(int(region[1]), int(region[2])-length+1):
        if 'N' in chr_seq[i:i+length]:
            continue
        nb_seq[i] = chr_seq[i:i+length]

In [21]:
nb_seq
# len(nb_seq)

{237550: 'TTCTTGGATGTTCGGG',
 237551: 'TCTTGGATGTTCGGGA',
 237552: 'CTTGGATGTTCGGGAA',
 237553: 'TTGGATGTTCGGGAAA',
 237554: 'TGGATGTTCGGGAAAG',
 237555: 'GGATGTTCGGGAAAGG',
 237556: 'GATGTTCGGGAAAGGG',
 237557: 'ATGTTCGGGAAAGGGG',
 237558: 'TGTTCGGGAAAGGGGG',
 237559: 'GTTCGGGAAAGGGGGT',
 237560: 'TTCGGGAAAGGGGGTT',
 237561: 'TCGGGAAAGGGGGTTA',
 237562: 'CGGGAAAGGGGGTTAT',
 237563: 'GGGAAAGGGGGTTATT',
 237564: 'GGAAAGGGGGTTATTA',
 237565: 'GAAAGGGGGTTATTAT',
 237566: 'AAAGGGGGTTATTATC',
 237567: 'AAGGGGGTTATTATCT',
 237568: 'AGGGGGTTATTATCTA',
 237569: 'GGGGGTTATTATCTAG',
 237570: 'GGGGTTATTATCTAGG',
 237571: 'GGGTTATTATCTAGGA',
 237572: 'GGTTATTATCTAGGAT',
 237573: 'GTTATTATCTAGGATC',
 237574: 'TTATTATCTAGGATCC',
 237575: 'TATTATCTAGGATCCT',
 237576: 'ATTATCTAGGATCCTT',
 237577: 'TTATCTAGGATCCTTG',
 237578: 'TATCTAGGATCCTTGA',
 237579: 'ATCTAGGATCCTTGAA',
 237580: 'TCTAGGATCCTTGAAG',
 237581: 'CTAGGATCCTTGAAGC',
 237582: 'TAGGATCCTTGAAGCA',
 237583: 'AGGATCCTTGAAGCAC',
 237584: 'GGAT

## 2. Calculating DNA physical properties of each sequence

We consider MGW. Read the two files into separate dictionaries

In [22]:
binding_mgw={}
max_mgw=0
with open('data/chr1_PAX5_b.txt', 'r') as file:
    next(file)
    for line in file:
        nt = line.rstrip().split('\t')
        binding_mgw[int(nt[1])] = float(nt[3])  
        if float(nt[3]) > max_mgw:
            max_mgw=float(nt[3])

In [23]:
binding_mgw

{710609: 2.85,
 710610: 4.36,
 710611: 5.57,
 710612: 5.82,
 710613: 5.0,
 710614: 4.53,
 710615: 5.43,
 710616: 5.4,
 710617: 4.94,
 710618: 4.77,
 710619: 5.08,
 710620: 5.25,
 710621: 5.71,
 710622: 5.94,
 710623: 5.15,
 710624: 4.31,
 762858: 4.78,
 762859: 4.89,
 762860: 5.8,
 762861: 5.4,
 762862: 4.77,
 762863: 4.7,
 762864: 5.22,
 762865: 5.65,
 762866: 5.2,
 762867: 5.02,
 762868: 5.15,
 762869: 5.27,
 762870: 5.08,
 762871: 4.8,
 762872: 4.18,
 762873: 3.75,
 805268: 4.96,
 805269: 5.02,
 805270: 5.66,
 805271: 5.74,
 805272: 5.25,
 805273: 5.08,
 805274: 4.96,
 805275: 4.73,
 805276: 4.78,
 805277: 4.93,
 805278: 4.85,
 805279: 4.95,
 805280: 5.3,
 805281: 5.51,
 805282: 5.51,
 805283: 5.37,
 840170: 5.05,
 840171: 4.63,
 840172: 4.8,
 840173: 5.18,
 840174: 5.46,
 840175: 5.42,
 840176: 5.54,
 840177: 5.41,
 840178: 5.09,
 840179: 5.37,
 840180: 5.29,
 840181: 5.09,
 840182: 5.37,
 840183: 5.29,
 840184: 4.77,
 840185: 4.85,
 894704: 4.63,
 894705: 4.73,
 894706: 4.76,
 894

In [24]:
b_seq_mgw={}
length=len(next(iter(b_seq.values())))

for key in b_seq.keys():
    mgw_seq=[]
    incomplete=False
    for i in range(key, key+length):
        val=binding_mgw.get(i, None)
        if val == None:
            incomplete=True
            break
        mgw_seq.append(val)
    if incomplete:
        continue
    b_seq_mgw[key]=mgw_seq

In [25]:
b_seq_mgw

{710609: [2.85,
  4.36,
  5.57,
  5.82,
  5.0,
  4.53,
  5.43,
  5.4,
  4.94,
  4.77,
  5.08,
  5.25,
  5.71,
  5.94,
  5.15,
  4.31],
 762858: [4.78,
  4.89,
  5.8,
  5.4,
  4.77,
  4.7,
  5.22,
  5.65,
  5.2,
  5.02,
  5.15,
  5.27,
  5.08,
  4.8,
  4.18,
  3.75],
 805268: [4.96,
  5.02,
  5.66,
  5.74,
  5.25,
  5.08,
  4.96,
  4.73,
  4.78,
  4.93,
  4.85,
  4.95,
  5.3,
  5.51,
  5.51,
  5.37],
 840170: [5.05,
  4.63,
  4.8,
  5.18,
  5.46,
  5.42,
  5.54,
  5.41,
  5.09,
  5.37,
  5.29,
  5.09,
  5.37,
  5.29,
  4.77,
  4.85],
 894704: [4.63,
  4.73,
  4.76,
  4.94,
  5.3,
  5.29,
  4.96,
  4.82,
  4.94,
  5.3,
  5.29,
  5.09,
  5.37,
  5.51,
  5.54,
  5.41],
 894719: [5.41,
  5.06,
  5.46,
  5.64,
  5.25,
  5.08,
  4.96,
  4.96,
  5.39,
  5.53,
  4.95,
  4.83,
  5.43,
  5.65,
  4.93,
  4.78],
 894833: [4.85,
  4.93,
  4.85,
  4.95,
  5.3,
  5.15,
  5.02,
  5.2,
  5.65,
  5.43,
  5.09,
  5.41,
  5.41,
  4.96,
  4.95,
  5.19],
 936172: [4.77,
  4.82,
  4.94,
  5.3,
  5.59,
  5.67,

In [30]:
nonbinding_mgw={}
with open('data/chr1_PAX5_nb.txt', 'r') as file:
    next(file)
    for line in file:
        nt = line.rstrip().split('\t')
        nonbinding_mgw[int(nt[1])] = float(nt[3])
        if float(nt[3]) > max_mgw:
            max_mgw=float(nt[3])

In [31]:
nonbinding_mgw

{237550: 4.46,
 237551: 4.85,
 237552: 4.8,
 237553: 4.92,
 237554: 5.31,
 237555: 5.42,
 237556: 4.94,
 237557: 4.84,
 237558: 4.82,
 237559: 4.3,
 237560: 4.85,
 237561: 5.1,
 237562: 5.38,
 237563: 5.22,
 237564: 4.95,
 237565: 4.73,
 237566: 4.63,
 237567: 4.36,
 237568: 4.05,
 237569: 4.42,
 237570: 4.62,
 237571: 4.75,
 237572: 4.58,
 237573: 4.03,
 237574: 4.87,
 237575: 5.62,
 237576: 5.53,
 237577: 5.11,
 237578: 5.28,
 237579: 5.89,
 237580: 4.72,
 237581: 4.74,
 237582: 5.84,
 237583: 5.69,
 237584: 4.98,
 237585: 4.51,
 237586: 4.63,
 237587: 4.63,
 237588: 4.51,
 237589: 4.31,
 237590: 4.65,
 237591: 5.4,
 237592: 6.0,
 237593: 5.15,
 237594: 4.82,
 237595: 4.61,
 237596: 5.43,
 237597: 5.56,
 237598: 4.77,
 237599: 4.58,
 237600: 4.75,
 237601: 4.89,
 237602: 5.44,
 237603: 5.31,
 237604: 4.65,
 237605: 4.42,
 237606: 4.73,
 237607: 5.02,
 237608: 5.33,
 237609: 5.38,
 237610: 4.66,
 237611: 3.9,
 237612: 4.52,
 237613: 4.5,
 237614: 4.67,
 237615: 5.29,
 237616: 5.9,
 23

In [32]:
nb_seq_mgw={}
length=len(next(iter(b_seq.values())))

for key in nb_seq.keys():
    mgw_seq=[]
    incomplete=False
    for i in range(key, key+length):
        val=nonbinding_mgw.get(i, None)
        if val == None:
            incomplete=True
            break
        mgw_seq.append(val)
    if incomplete:
        continue
    nb_seq_mgw[key]=mgw_seq

In [33]:
nb_seq_mgw

{237550: [4.46,
  4.85,
  4.8,
  4.92,
  5.31,
  5.42,
  4.94,
  4.84,
  4.82,
  4.3,
  4.85,
  5.1,
  5.38,
  5.22,
  4.95,
  4.73],
 237551: [4.85,
  4.8,
  4.92,
  5.31,
  5.42,
  4.94,
  4.84,
  4.82,
  4.3,
  4.85,
  5.1,
  5.38,
  5.22,
  4.95,
  4.73,
  4.63],
 237552: [4.8,
  4.92,
  5.31,
  5.42,
  4.94,
  4.84,
  4.82,
  4.3,
  4.85,
  5.1,
  5.38,
  5.22,
  4.95,
  4.73,
  4.63,
  4.36],
 237553: [4.92,
  5.31,
  5.42,
  4.94,
  4.84,
  4.82,
  4.3,
  4.85,
  5.1,
  5.38,
  5.22,
  4.95,
  4.73,
  4.63,
  4.36,
  4.05],
 237554: [5.31,
  5.42,
  4.94,
  4.84,
  4.82,
  4.3,
  4.85,
  5.1,
  5.38,
  5.22,
  4.95,
  4.73,
  4.63,
  4.36,
  4.05,
  4.42],
 237555: [5.42,
  4.94,
  4.84,
  4.82,
  4.3,
  4.85,
  5.1,
  5.38,
  5.22,
  4.95,
  4.73,
  4.63,
  4.36,
  4.05,
  4.42,
  4.62],
 237556: [4.94,
  4.84,
  4.82,
  4.3,
  4.85,
  5.1,
  5.38,
  5.22,
  4.95,
  4.73,
  4.63,
  4.36,
  4.05,
  4.42,
  4.62,
  4.75],
 237557: [4.84,
  4.82,
  4.3,
  4.85,
  5.1,
  5.38,
  5.

In [26]:
import random

keyset=random.sample(nb_seq.keys(), int(ratio*len(nb_seq)))

## 3.1 Converting experimental sequence data into form that a Classifier can accept

We do this by converting binding information into a vector of nucleotides. This will require the sequence of chromosome 1, which we have. In addition, we will sample regulatory regions without binding sites to build a sample of non-binding data points. 

Note: if a TF binds on the negative strand, we will reverse the DNA sequence and take the complement.

In [27]:
letters={'A': 0, 'C': 0, 'G': 0, 'T': 0}
for seq in b_seq.values():
    for letter in seq:
        letters[letter]+=1

In [28]:
total = 0
for letter in letters:
    total += letters[letter]

for letter in letters:
    letters[letter] /= total

In [29]:
# nucleotide distribution in the binding sites
letters

{'A': 0.1611294240111034,
 'C': 0.3253383067314365,
 'G': 0.4491672449687717,
 'T': 0.06436502428868841}

In [30]:
# convert sequences to a form that can be accepted a machine classifier

X_b=[]

nt_ind={'A':0, 'C':1, 'G':2, 'T':3}
#print(sorted(to_app.keys()))
for key in b_seq_mgw.keys():
    seq=b_seq[key]

    x_seq = [[],[],[],[]]
    x=[]
    for nt in seq:
        x_seq[nt_ind[nt]].append(1)
        for i in range(4):
            if i == nt_ind[nt]:
                continue
            x_seq[i].append(0)
    for i in x_seq:
        x.extend(i)
    if physical:
        for i in b_seq_mgw[key]:
            x.append(i/max_mgw)
    X_b.append(x)

In [31]:
X_b[0]

[1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0]

## 3.2 Generate set of non-binding regions for classification

Generate every single sequence 15 nt sequence (with overlaps) from the non-binding list and randomly select 15k of them.

In [32]:
X_nb=[]
y_nb=[]

nt_ind={'A':0, 'C':1, 'G':2, 'T':3}
#print(sorted(to_app.keys()))
for key in keyset:
    seq=nb_seq[key]

    x_seq = [[],[],[],[]]
    x=[]
    for nt in seq:
        x_seq[nt_ind[nt]].append(1)
        for i in range(4):
            if i == nt_ind[nt]:
                continue
            x_seq[i].append(0)
    for i in x_seq:
        x.extend(i)
    if physical:
        for i in nb_seq_mgw[key]:
            x.append(i/max_mgw)
    X_nb.append(x)
    y_nb.append(0)

In [33]:
X_nb[0]

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

## 4. Train a Machine Learning Classifier

1. Start by training a classifier using just sequence data. Then use a similar technique but incorporate physical data as well. See if there is any difference.

In [34]:
# baseline classifier without considering DNA physical properties

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X=[]
X.extend(X_b)
X.extend(X_nb)

y=[]
y.extend(y_b)
y.extend(y_nb)

X_other, X_valid, y_other, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_other, y_other, test_size=0.25, random_state=42)

In [ ]:
svc=SVC(kernel='linear')
svc.fit(X_train, y_train)

In [330]:
test_pred=svc.predict(X_test)

In [84]:
from sklearn import metrics

print(metrics.f1_score(y_test, test_pred))

NameError: name 'test_pred' is not defined

## 5. Use position weight matrix as another baseline

In [35]:
# dictionary with TF name as key and matrix as value
pwm={}

with open('data/factorbookMotifPwm.txt', 'r') as file:
    for line in file:
        values=line.rstrip().split('\t')
        matrix=[]
        A=values[2].rstrip().split(',')
        C=values[3].rstrip().split(',')
        G=values[4].rstrip().split(',')
        T=values[5].rstrip().split(',')
        print(A)
        matrix.extend(A[:-1])
        matrix.extend(C[:-1])
        matrix.extend(G[:-1])
        matrix.extend(T[:-1])
        lst=[float(i) for i in matrix]
        pwm[values[0]]=lst

['0.137931', '0.620690', '0.655172', '0.000000', '0.112069', '0.818966', '0.000000', '0.000000', '0.344828', '0.431034', '0.318966', '0.000000', '0.017241', '1.000000', '0.094828', '0.258621', '0.172414', '0.327586', '0.189655', '0.172414', '0.051724', '']
['0.336842', '0.240000', '0.387368', '0.193684', '0.298947', '0.328421', '0.000000', '0.000000', '1.000000', '0.056842', '0.006316', '0.046316', '1.000000', '0.008421', '']
['0.160896', '0.221996', '0.272912', '0.299389', '0.201629', '0.144603', '0.010183', '0.000000', '0.000000', '0.030550', '0.083503', '0.690428', '0.016293', '0.000000', '0.006110', '']
['0.086957', '0.000000', '0.000000', '0.000000', '0.630435', '1.000000', '0.000000', '0.010870', '0.076087', '0.000000', '0.000000', '']
['0.091584', '0.086634', '0.004950', '0.544554', '0.198020', '0.000000', '0.000000', '0.007426', '1.000000', '0.952970', '0.002475', '0.019802', '0.113861', '0.259901', '0.029703', '0.188119', '']
['0.193347', '0.112266', '1.000000', '0.000000', '0

In [36]:
def calcScore(sequence, matrix):
    score=1
    if len(sequence) != len(matrix):
        print("This sequence does not correspond with the given PWM.")
        return
    for nt, val in zip(sequence, matrix):
        if nt == 1:
            score *= val
    return score

# we need to learn a threshold score - use the min of the scores obtained by binding sequences
least_pos=1
def pwmLearn(sequences, labels, protein):
    global least_pos
    for seq, label in zip(sequences, labels):
        if label == 0:
            continue
        else:
            score=calcScore(seq, pwm[protein])
            if score < least_pos:
                least_pos = score

def pwmPredict(sequences, protein):
    output=[]
    for seq in sequences:
        if calcScore(seq, pwm[protein]) > least_pos:
            output.append(1)
        else:
            output.append(0)
    return output

In [37]:
pwmLearn(X_train, y_train, 'PAX5')

In [38]:
least_pos

6.799834151090865e-08

In [39]:
k=pwmPredict(X_test, 'PAX5')

In [41]:
from sklearn import metrics
print(metrics.f1_score(y_test, k))

0.6454265159301129
